# Reddit Flair Detector

In [0]:
# to avoid the warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle
import cv2 as cv
import matplotlib.pyplot as plt

# 1. Data Acquisation and Preprocessing

## 1.1 Scrapping subreddit r/india using praw and pushshift.io

In [0]:
# !pip install praw

In [0]:
import praw

In [0]:
import requests
import json
import csv
import time
import datetime

In [0]:
# create an app on and then an instance of Reddit App: https://ssl.reddit.com/prefs/apps/ 
reddit = praw.Reddit(client_id='xVeb5-ej49aBDg', client_secret='HLuuBB0e1upw1pbZ-oUFtrBplFY', user_agent='reddit-scrap', username='macabdul9', password='Sudo$0#1')

In [0]:
# get the user
print(reddit.user.me())

macabdul9


In [0]:
subm = reddit.submission(url='https://www.reddit.com/r/india/comments/d0ytzc/nasa_on_twitter_space_is_hard_we_commend_isro_s/')

In [0]:
subm1 = reddit.submission(id='d0ytzc')

In [0]:
subm.permalink

'/r/india/comments/d0ytzc/nasa_on_twitter_space_is_hard_we_commend_isro_s/'

In [0]:
subm1.title

'NASA on Twitter: Space is hard. We commend @ISRO ’s attempt to land their #Chandrayaan2 mission on the Moon’s South Pole. You have inspired us with your journey and look forward to future opportunities to explore our solar system together.'

### 1.1.1 Scrapping the *submissions*

In [0]:
def getPushshiftSubData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [0]:
def collectSubData(subm):
    subData = list() #list to store data points
    
    sub_id = subm['id']
    
    sub_text = subm['title']
    
    author = subm['author']
    
    full_link = subm['full_link']
    
    permalink = subm['permalink']
    
    url = subm['url']
    
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    
    
    score = subm['score']
    
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    
    numComms = subm['num_comments']
    
    
    over_18 = subm['over_18']
    
    
    subData.append((sub_id, sub_text, author, full_link, permalink, url, flair, score, created, numComms, over_18))
    
    subStats[sub_id] = subData
    

In [0]:
#Subreddit to query
sub='india'

#before and after dates
before = "1567209600" # Sat, 31 Aug 2019 00:00:00 
after = "1546300800"  # Tue, 01 Jan 2019 00:00:00 

query = "" # title should have either null string more 

subCount = 0
subStats = {}

In [0]:
# properties_of_a_submission = submission

In [0]:
data = getPushshiftSubData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftSubData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1546300800&before=1567209600&subreddit=india
1000
2019-01-03 02:43:10
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1546483390&before=1567209600&subreddit=india
1000
2019-01-04 17:19:53
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1546622393&before=1567209600&subreddit=india
1000
2019-01-07 05:26:38
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1546838798&before=1567209600&subreddit=india
1000
2019-01-09 03:07:34
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1547003254&before=1567209600&subreddit=india
1000
2019-01-10 17:16:56
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1547140616&before=1567209600&subreddit=india
1000
2019-01-12 19:27:51
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1547321271&before=1567209600&subreddit=india
1000
2019-01-15 0

In [0]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

128123 submissions have added to list
1st entry is:
[P] New Year smiles: What do you say to get Mukesh Ambani’s attention? ‘Sir, Jio chal nahi raha hai’ created: https://scroll.in/article/907250/new-year-smiles-what-do-you-say-to-get-mukesh-ambanis-attention-sir-jio-chal-nahi-raha-hai?utm_source=rss&amp;utm_medium=public
Last entry is:
A cousin who is worried about "loved jihad" just sent me some joke meme about how we can all go marry Kashmiri women now. How do you deal with people like this? created: https://www.reddit.com/r/india/comments/cxovsz/a_cousin_who_is_worried_about_loved_jihad_just/


In [0]:
def updateSubs_file():
    upload_count = 0
    
    print("input filename of submission file, please add .csv")
    filename = input()
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["sub_id", "sub_text", "author", "full_link", "permalink", "url", "flair", "score", "created", "numComms", "over_18"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")

In [0]:
updateSubs_file()

input filename of submission file, please add .csv
india_subreddit_2019_submissions.csv
128123 submissions have been uploaded


### 1.1.1 Scrapping the comments

In [0]:
def getPushshiftCommentData(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/comment/?q='+str(query)+'&subreddit=india'+'&size=1000&after='+str(after)+'&before='+str(before)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [0]:
def collectCommentData(comment):
  
    comData = list() #list to store data points
    
    com_id = comment['id']
    
    com_text = comment['body']
    
    subm = comment['link_id'].split('_')[-1] #reddit.comment(id=com_id).submission
    
#     subm = comment['submission']
    
    comData.append((com_id, com_text, subm))
    
    comStats[com_id] = comData
    

In [0]:
#Subreddit to query
subreddit='india'

#before and after dates
before = "1567209600" # Sat, 31 Aug 2019 00:00:00 
after = "1546300800"  # Tue, 01 Jan 2019 00:00:00 

query = "" # title should have either null string more 

comCount = 0
comStats = {}

In [0]:
# comData = getPushshiftCommentData(query, after, before)# Will run until all comments have been gathered 

# # from the 'after' date up until before date
# while len(comData) > 0:
#     for comment in comData:
#         collectCommentData(comment)
#         comCount+=1
        
#     # Calls getPushshiftCommentData() with the created date of the last comment
#     print(len(comData))
#     print(str(datetime.datetime.fromtimestamp(comData[-1]['created_utc'])))
#     after = comData[-1]['created_utc']
#     data =getPushshiftCommentData(query, after, before)

# print(len(comData))


## 1.2 Processing the scrapped data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving india_subreddit_2019_submissions_rectified.csv to india_subreddit_2019_submissions_rectified.csv


In [0]:
rectified_df = pd.read_csv('india_subreddit_2019_submissions_rectified.csv')

In [0]:
rectified_df.tail()

,Unnamed: 0,sub_id,sub_text,author,full_link,permalink,url,flair,score,created,numComms,over_18
57554,128115,cxohgh,Garam Hawa - Film (Movie) Plot and Review,choicelesspawn,https://www.reddit.com/r/india/comments/cxohgh...,/r/india/comments/cxohgh/garam_hawa_film_movie...,http://www.filmreference.com/Films-Fr-Go/Garam...,[R]eddiquette,17,2019-08-30 23:14:01,5,False
57555,128116,cxoom2,Samba Natthu Cheran Pandiyan 1080p HD Video Song,GadgetsHomeUSA,https://www.reddit.com/r/india/comments/cxoom2...,/r/india/comments/cxoom2/samba_natthu_cheran_p...,https://sangeetdunia.com/samba-natthu-cheran-p...,[R]eddiquette,1,2019-08-30 23:31:33,0,False
57556,128117,cxopqf,"""Growth in free fall, ugly economic data, Modi...",wrtjhragsh,https://www.reddit.com/r/india/comments/cxopqf...,/r/india/comments/cxopqf/growth_in_free_fall_u...,https://youtu.be/RIX5OYR1Yj4,[R]eddiquette,5,2019-08-30 23:34:14,0,False
57557,128118,cxopqt,Do you perceive your country a good choice for...,DigitalNomadDE,https://www.reddit.com/r/india/comments/cxopqt...,/r/india/comments/cxopqt/do_you_perceive_your_...,https://www.reddit.com/r/india/comments/cxopqt...,AskIndia,15,2019-08-30 23:34:16,9,False
57558,128119,cxos69,"Indian-Origin Man, Trump's Former Partner, Arr...",artichokepantherss,https://www.reddit.com/r/india/comments/cxos69...,/r/india/comments/cxos69/indianorigin_man_trum...,https://www.ndtv.com/indians-abroad/indian-ori...,Non-Political,1,2019-08-30 23:40:23,0,False


In [0]:
rectified_df.shape

(57559, 12)

In [0]:
titles = rectified_df['sub_text']
flairs = rectified_df['flair']
ids = rectified_df['sub_id']

In [0]:
unique_flairs = set(flairs)

In [0]:
len(unique_flairs)

11

In [0]:
flairs_freq = np.unique(np.array(flairs, dtype=np.dtype('U100')), return_counts=True)

In [0]:
flairs_freq_dict = dict()

for i in range(len(flairs_freq[0])):
  if flairs_freq[1][i] > 700 and flairs_freq[0][i] != 'nan':
    flairs_freq_dict[flairs_freq[0][i]] = flairs_freq[1][i]

In [0]:
flairs_freq_dict

{'AskIndia': 9907,
 'Business/Finance': 4776,
 'Food': 855,
 'Non-Political': 15019,
 'Not in English.': 781,
 'Photography': 1865,
 'Policy/Economy': 2141,
 'Politics': 15449,
 'Science/Technology': 2830,
 'Sports': 1362,
 '[R]eddiquette': 2574}

In [0]:
len(flairs_freq_dict)

11

In [0]:
sum(flairs_freq_dict.values())

57559

In [0]:
cols = rectified_df.columns

In [0]:
keys_flairs = dict(zip(list(rectified_df.sub_id), list(rectified_df.flair)))

In [0]:
with open('keys_flairs.pkl', 'wb') as fp:
  pickle.dump(keys_flairs, fp)

In [0]:
rectified_df.shape[0]

57559

In [0]:
print(len(titles))
print(len(flairs))
print(len(ids))

57559
57559
57559


In [0]:
idx_to_flair = dict(zip(list(np.linspace(start=0, stop=10, num=11, dtype=int)), list(flairs_freq_dict.keys())))
flair_to_idx = dict([(value, key) for (key, value) in idx_to_flair.items()])

In [0]:
idx_to_flair

{0: 'AskIndia',
 1: 'Business/Finance',
 2: 'Food',
 3: 'Non-Political',
 4: 'Not in English.',
 5: 'Photography',
 6: 'Policy/Economy',
 7: 'Politics',
 8: 'Science/Technology',
 9: 'Sports',
 10: '[R]eddiquette'}

In [0]:
with open('idx_to_flair.pkl', 'wb') as fp:
  pickle.dump(idx_to_flair, fp)
  
with open('flair_to_idx.pkl', 'wb') as fp:
  pickle.dump(flair_to_idx, fp)

In [0]:
type(titles)

pandas.core.series.Series

In [0]:
titles[12]

'Happy new year FIREWORKS AT TOP10 SKYSCRAPERS OF WORLD'

In [0]:
flairs[10:20]

10     Non-Political
11    Policy/Economy
12       Photography
13          AskIndia
14          Politics
15          Politics
16          AskIndia
17          AskIndia
18            Sports
19       Photography
Name: flair, dtype: object

## 1.3 Text Preprocessing

In [0]:
import nltk
from nltk.tokenize  import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def clean_view(text):
    
    
    stopword = set(stopwords.words('english'))
    
    text = text.lower()
    
    tokenizer = RegexpTokenizer('[0-9a-z]+')
    

    
    tokens = tokenizer.tokenize(text)
    
    new_token = [token for token in tokens if token not in stopword] # stemming and stopword removing
    
    return ' '.join(tokens)
  

In [0]:
clean_sub_text = rectified_df['sub_text'].apply(clean_view)

In [0]:
titles[1001]

'AAP, Cong to all for 2019 polls in Delhi: Ex-AAP leader Ashutosh'

In [0]:
clean_sub_text[1001]

'aap cong to all for 2019 polls in delhi ex aap leader ashutosh'

### 1.4 LSTM Modeling

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(clean_sub_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [0]:
# unique_words = set()

In [0]:
for title in clean_sub_text:
  unique_words.update(title.split(" "))

In [0]:
len(unique_words)

37625

In [0]:
# almost same

In [0]:
# Truncate and pad the input sequences so that they are all in the same length for modeling.

In [0]:
tokenizer2 = Tokenizer()

In [0]:
tokenizer2.word_counts = tokenizer.word_counts
tokenizer2.word_docs = tokenizer.word_docs
tokenizer2.word_index = tokenizer.word_index

In [0]:
with open('word_counts.pkl', 'wb') as fp:
  pickle.dump(tokenizer.word_counts, fp)


In [0]:
with open('word_docs.pkl', 'wb') as fp:
  pickle.dump(tokenizer.word_docs, fp)


In [0]:
with open('word_index.pkl', 'wb') as fp:
  pickle.dump(tokenizer.word_index, fp)

In [0]:
with open('word_counts.pkl', 'rb') as fp:
  word_counts = pickle.load(fp)
with open('word_docs.pkl', 'rb') as fp:
  word_docs = pickle.load(fp)
with open('word_index.pkl', 'rb') as fp:
  word_index = pickle.load(fp)

In [0]:
tokenizer3 = Tokenizer()

In [0]:
tokenizer3.word_counts = word_counts
tokenizer3.word_docs = word_docs
tokenizer3.word_index = word_index

In [0]:
a = tokenizer2.texts_to_sequences(clean_sub_text.values[0])
b = tokenizer.texts_to_sequences(clean_sub_text.values[0])
c = tokenizer3.texts_to_sequences(clean_sub_text.values[0])

In [0]:
import re

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

In [0]:
stopwords = [ 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 
'there', 'about', 'once', 'during', 'out', 'very', 'having', 
'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its',
'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off',
'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the',
'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his',
'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this',
'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to',
'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them',
'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves',
'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 
'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 
'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 
'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 
'here', 'than']

In [0]:
def text_preprocessing(text):
    
    
#     stopword = set(stopwords.words('english'))

    text = text.lower()

    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.


    

    tokens = [token for token in text.split() if token not in stopwords] # stemming and stopword removing

    return " ".join(tokens)

In [0]:
text = 'I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.'

In [0]:
len(text)

229

In [0]:
processed_text = text_preprocessing(text)

In [0]:
processed_text

'victim identity theft someone stole identity personal information open visa credit card account bank america following bank america visa credit card account belong xxxx'

In [0]:
clean_sub_text = rectified_df['sub_text'].apply(text_preprocessing)

In [0]:
type(clean_sub_text)

pandas.core.series.Series

In [0]:
x = tokenizer.texts_to_sequences(processed_text)

In [0]:
x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
x.shape

(226, 100)

In [0]:
X = tokenizer.texts_to_sequences(clean_sub_text.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (57559, 100)


In [0]:
X[0].shape

(100,)

In [0]:
type(flairs)

pandas.core.series.Series

In [0]:
# Converting categorical labels to numbers.
Y = pd.get_dummies(flairs).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (57559, 11)


In [0]:
# train-text split
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(51803, 100) (51803, 11)
(5756, 100) (5756, 11)


### 1.4.1   LSTM Architecture
- The first layer is the embedded layer that uses 100 length vectors to represent each word.
- SpatialDropout1D performs variational dropout in NLP models.
- The next layer is the LSTM layer with 100 memory units.
- The output layer must create 12 output values, one for each class.
- Activation function is softmax for multi-class classification.
- Because it is a multi-class classification problem, categorical_crossentropy is used as the loss function.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.optimizers import *

In [0]:
epochs = 20
batch_size = 64

In [0]:
es =  EarlyStopping(monitor='val_acc', min_delta = 0.5, patience = 5)
mcp = ModelCheckpoint('reddit_flair_detector.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)

In [0]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64))
model.add(Dense(11, activation='softmax'))


In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          5000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_2 (Dense)              (None, 11)                715       
Total params: 5,042,955
Trainable params: 5,042,955
Non-trainable params: 0
_________________________________________________________________


In [0]:
# !pip install keras_radam
import keras_radam

In [0]:
radam = keras_radam.RAdam()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [0]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[es, mcp])

Train on 46622 samples, validate on 5181 samples
Epoch 1/20
46622/46622 [==============================] - 144s 3ms/step - loss: 1.4226 - acc: 0.5283 - val_loss: 1.2020 - val_acc: 0.6051
Epoch 2/20
46622/46622 [==============================] - 143s 3ms/step - loss: 0.9284 - acc: 0.6955 - val_loss: 1.2011 - val_acc: 0.6122
Epoch 3/20
46622/46622 [==============================] - 142s 3ms/step - loss: 0.6504 - acc: 0.7887 - val_loss: 1.3316 - val_acc: 0.5918
Epoch 4/20
46622/46622 [==============================] - 143s 3ms/step - loss: 0.4680 - acc: 0.8473 - val_loss: 1.5053 - val_acc: 0.5736
Epoch 5/20
46622/46622 [==============================] - 141s 3ms/step - loss: 0.3515 - acc: 0.8849 - val_loss: 1.7147 - val_acc: 0.5713
Epoch 6/20
46622/46622 [==============================] - 144s 3ms/step - loss: 0.2729 - acc: 0.9113 - val_loss: 1.9034 - val_acc: 0.5648


In [0]:
from google.colab import files
uploaded = files.upload()

Saving reddit_flair_detector(1).h5 to reddit_flair_detector(1).h5


In [0]:
model.load_weights('reddit_flair_detector(1).h5')

In [0]:
model.evaluate(x_train, y_train)

51803/51803 [==============================] - 112s 2ms/step


[0.6978255155714608, 0.7851282744346165]

In [0]:
model.evaluate(x_test, y_test)

5756/5756 [==============================] - 12s 2ms/step


[1.2173395002774683, 0.608756080528694]

In [0]:
x_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  234,  783,   41,
       2689], dtype=int32)

In [0]:
model.predict(x_train[0])

ValueError: ignored

In [0]:
x_test[0].shape

(100,)

In [0]:
X_ = clean_sub_text.values
Y_ = np.array([flair_to_idx.get(flair) for flair in flairs])

In [0]:
title_flair_df = pd.DataFrame(data=zip(X_, Y_), columns=['title', 'flair'])

In [0]:
title_flair_df.head()

,title,flair
0,palette pollution,3
1,allow banks hold passports loantakers madras h...,1
2,allow banks hold passports loantakers madras h...,1
3,tamil nadu usher new year green note ban singl...,6
4,heera gold timeline #1 background pre 2007,1


In [0]:
title_flair_df.to_csv('title_flair.csv', index=False)

In [0]:
with open('title_flair.pkl', 'wb') as fp:
  pickle.dump(title_flair_df, fp)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X_, Y_, test_size = 0.2)

In [0]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(46047,) (46047,)
(11512,) (11512,)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tf = TfidfVectorizer(
    analyzer='word', 
    sublinear_tf=True,
    strip_accents='unicode',
    ngram_range=(1, 1),
    max_features=10000)

In [0]:
tf.fit(X_)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

In [0]:
x_train = tf.transform(x_train)
x_test = tf.transform(x_test)

In [0]:
a = []
a.append('I am good boy')

In [0]:
a = tf.transform(a).toarray()

In [0]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(46047, 10000) (46047,)
(11512, 10000) (11512,)


In [64]:
type(x_test)

scipy.sparse.csr.csr_matrix

## 1.4 Classification Models

### 1.4.1 Naive Bayes Classifier

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
model = MultinomialNB()

In [0]:
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
model.score(x_train, y_train)

0.6178252654896085

In [0]:
print(x_train.shape)
print(x_test.shape)

(46047, 10000)
(11512, 10000)


In [0]:
model.score(x_test, y_test)

0.5515983321751217

### 2.1.2 Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lr = LogisticRegression()

In [0]:
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
lr.score(x_train, y_train)

0.6609985449649272

In [0]:
lr.score(x_test, y_test)

0.578092425295344

### Ensemble Method

In [0]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



In [0]:
classifiers = [
    ('knn', KNeighborsClassifier(11)),
    ('svc', SVC(kernel="rbf", C=0.025, probability=True)),
    ('Nuvc', NuSVC(probability=True)),
    ('dtree', DecisionTreeClassifier()),
    ('rf', RandomForestClassifier()),
    ('ada', AdaBoostClassifier()),
    ('gdbc', GradientBoostingClassifier()),
    ('gnb', GaussianNB()),
    ('lda', LinearDiscriminantAnalysis()),
    ('qda', QuadraticDiscriminantAnalysis())]

In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [0]:
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',  random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = MultinomialNB()


In [0]:
eclf1 = VotingClassifier(estimators=classifiers, voting='hard')

In [0]:
eclf1 = eclf1.fit(x_train, y_train)

In [0]:
eclf1.score(x_train, y_train)

0.7234347514496058

In [0]:
eclf1.score(x_test, y_test)

0.5823488533703961

### 2.1.3 Support Vector Machines

In [0]:
from sklearn.svm import SVC

In [0]:
svm = SVC()

In [0]:
svm.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
svm.score(x_train, y_train)

0.27822328302600746

In [0]:
svm.score(x_test, y_test)

0.2740714867854232

### 2.1.3 Random Forest Model


In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
forest = RandomForestClassifier(max_depth=100, n_estimators=500)

In [0]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
forest.score(x_train, y_train)

0.6504780893888897

In [0]:
forest.score(x_test, y_test)

0.5190941902433345

In [0]:
forest.score(x_train, y_train)

0.5186069188339567

In [0]:
forest.score(x_test, y_test)

0.47194085458144136

### Gathering more data

In [0]:
len(corresponding_ids)

85889

In [0]:
titles[18]

"[NP] [Ask] Will a xerox of my aadhar card with my college ID card work if I'm traveling via flight? Or is having the original aadhar card absolutely necessary?"

In [0]:
flairs[18]

'AskIndia'

In [0]:
type(titles[18])

str

In [0]:
titles_archive = titles

In [0]:
titles = titles_archive

In [0]:
titles[18]

"[NP] [Ask] Will a xerox of my aadhar card with my college ID card work if I'm traveling via flight? Or is having the original aadhar card absolutely necessary?"

In [0]:

# we will concate top 5 comments in title

for i in range(len(corresponding_ids)):
  
  subm = reddit.submission(id=corresponding_ids[i])
  comments = subm.comments
  
  count = 0
  
  for comment in comments:
    if count == 3:
      break
    titles[i] = titles[i] + "\n" + comment.body
    
    count += 1
  
  if i%1000 == 0:
    print("%d titles processed"%i)

0 titles processed
1000 titles processed
2000 titles processed
3000 titles processed
4000 titles processed
5000 titles processed


KeyboardInterrupt: ignored

In [0]:
titles = titles[:5000]
corresponding_ids = corresponding_ids[:5000]
flairs = flairs[:5000]

### I would like to address the issue faced by both techniques(praw reddit API wraper and pushshift.io) used for scrapping the data from India subreddit.
- praw is unable the search the data for more than 1000 submissions at a time 
- pushshift.io is unable to provide all the information we need to proceed further

### the solution to this is, I will used pushshift.io to scrap the submission id and praw for information gathering, It will be first of its kind.

In [0]:
ids = india_subreddit_df['Post ID']

In [0]:
len(ids)

192693

#### here we have 192693 submission ids now we will scrap data using praw

In [0]:
sub_dict = {}



In [0]:
submission = reddit.submission(id=ids[89])


In [0]:
comments = submission.comments

i = 0

c = []

for comment in comments:
  if i < 5:
#     print(comment.body)
      c.append(comment.body)

In [0]:
submission.num_comments

0

In [0]:
c[1]

'And they will be wondering why it’s not selling and that poor sales manager will be under fire. If this is done by the employee on purpose - it’s either sweet revenge or malicious compliance. '

In [0]:
submission.link_flair_text

'Low-effort Post.'